In [6]:
import pandas as pd

# Adjust pandas options to see all columns and to read subgroup descriptions fully
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [15]:
df_s = pd.read_csv("results/subgroup_model_results.csv")
#df_s.columns

In [16]:
# Assign rank: one per row (since you now have only subgroup_model rows)
df_s['subgroup_rank'] = (df_s.index + 1).astype("object")
df_s.loc[df_s['model_type'] == 'global', 'subgroup_rank'] = "N/A"

# Round columns for better readability
for col, ndigits in [('cookD', 2), ('r2', 3), ('mae', 3), ('mse', 3), ('mean_residual', 3),
                     ('global_baseline_r2', 3), ('global_baseline_mae', 3), ('global_baseline_mse', 3), ('global_baseline_mean_residual', 3),
                     ('mean_baseline_r2', 3), ('mean_baseline_mae', 3), ('mean_baseline_mse', 3), ('mean_baseline_mean_residual', 3)]:
    if col in df_s.columns:
        df_s[col] = df_s[col].round(ndigits)

# Compare r2 between subgroup_model and global baseline for each row
df_s['subgroup_r2_better'] = (df_s['r2'] > df_s['global_baseline_r2']).astype("object")
df_s.loc[df_s['model_type'] == 'global', 'subgroup_r2_better'] = None
df_s['subgroup_better_global'] = (df_s['ttest_p'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_global'] = None
df_s['subgroup_better_mean'] = (df_s['ttest_p_mean'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_mean'] = None
df_s['global_better_mean'] = (df_s['ttest_p_mean_global'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'global_better_mean'] = None

# Add decision column based on the rules
def determine_decision(row):
    sg = row['subgroup_better_global']
    sm = row['subgroup_better_mean']
    gm = row['global_better_mean']
    
    if sg and sm:
        return "True discovery"
    elif sg and not sm:
        return "True discovery, but model not appropriate"
    elif not sg and sm:
        return "Weak discovery"
    elif not sg and not sm and gm:
        return "False discovery"
    else:
        return "Model not appropriate"

df_s['decision'] = df_s.apply(determine_decision, axis=1)

# Specify desired column order
main_cols = [
    'subgroup_rank', 'model_type', 'description', 'decision', 'subgroup_r2_better', 'cookD', 'n_train', 'n_test',
    'r2', 'global_baseline_r2', 'mean_baseline_r2', 
    'mae', 'global_baseline_mae', 'mean_baseline_mae',
    'mse', 'global_baseline_mse', 'mean_baseline_mse',
    'mean_residual', 'global_baseline_mean_residual', 'mean_baseline_mean_residual'
]
rest_cols = [c for c in df_s.columns if c not in main_cols]
new_order = main_cols + rest_cols

# Reorder columns
df_s = df_s[new_order]

C:\Users\danic\AppData\Local\Temp\ipykernel_21068\2205722591.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_global'] = None
C:\Users\danic\AppData\Local\Temp\ipykernel_21068\2205722591.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_mean'] = None
C:\Users\danic\AppData\Local\Temp\ipykernel_21068\2205722591.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible

In [17]:
# Check global model
df_s.tail(1)

,subgroup_rank,model_type,description,decision,subgroup_r2_better,cookD,n_train,n_test,r2,global_baseline_r2,mean_baseline_r2,mae,global_baseline_mae,mean_baseline_mae,mse,global_baseline_mse,mean_baseline_mse,mean_residual,global_baseline_mean_residual,mean_baseline_mean_residual,intercept,coef__nr_distinct_files_viewed,pval__nr_distinct_files_viewed,coef__total_course_activities,pval__total_course_activities,coef__nightly_activities,pval__nightly_activities,coef__distinct_days,pval__distinct_days,coef__logged_in_weekly,pval__logged_in_weekly,coef__nr_files_viewed,pval__nr_files_viewed,coef__nr_slides_viewed,pval__nr_slides_viewed,ttest_p,ttest_stat,wilcoxon_p,wilcoxon_stat,ttest_p_mean,ttest_stat_mean,wilcoxon_p_mean,wilcoxon_stat_mean,ttest_p_mean_global,ttest_stat_mean_global,wilcoxon_p_mean_global,wilcoxon_stat_mean_global,subgroup_better_global,subgroup_better_mean,global_better_mean
100,N/A,global,NaN,True discovery,None,NaN,1552,388,0.11,NaN,-0.0,1.598,NaN,1.685,3.693,NaN,4.151,0.012,NaN,0.029,4.739552,0.032245,1.285434e-10,0.001919,0.06841,0.00047,0.589183,0.004481,0.615731,-0.246608,0.026818,-0.002626,0.569904,-0.011127,0.000892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Check subgroup models
df_s[['description', 'decision',   
      'r2', 'global_baseline_r2', 'mean_baseline_r2',
      'mse', 'global_baseline_mse', 'mean_baseline_mse',
      'ttest_p', 'n_test', 'subgroup_better_global', 'subgroup_better_mean', 'global_better_mean']].head(100)

,description,decision,r2,global_baseline_r2,mean_baseline_r2,mse,global_baseline_mse,mean_baseline_mse,ttest_p,n_test,subgroup_better_global,subgroup_better_mean,global_better_mean
0,course_repeater==np.False_ ∧ croho=='B Electrical Engineering' ∧ origin=='ORIGIN_N',Model not appropriate,0.038,0.025,-0.003,3.344,3.388,3.486,0.473959,27,False,False,False
1,course_repeater==np.False_ ∧ croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E',Model not appropriate,-0.015,-0.158,-0.033,4.400,5.021,4.480,0.269809,19,False,False,False
2,croho=='B Electrical Engineering' ∧ origin=='ORIGIN_N' ∧ type_vooropleiding=='VWO',Model not appropriate,0.047,-0.089,-0.019,2.591,2.959,2.770,0.288267,24,False,False,False
3,croho=='B Computer Science & Engineering' ∧ origin=='ORIGIN_E' ∧ sex=='SEX_M',True discovery,0.201,-0.410,-0.094,3.122,5.508,4.273,0.007214,21,True,True,False
4,course_repeater==np.False_ ∧ croho=='B Electrical Engineering' ∧ type_vooropleiding=='BUITENL_SL',Model not appropriate,-0.251,-0.007,-0.006,5.209,4.192,4.191,0.864519,43,False,False,False
5,croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E',Model not appropriate,0.018,-0.082,-0.023,4.146,4.566,4.317,0.281007,21,False,False,False
6,croho=='B Electrical Engineering' ∧ double_major==False ∧ origin=='ORIGIN_E',Model not appropriate,0.018,-0.082,-0.023,4.146,4.566,4.317,0.281007,21,False,False,False
7,croho=='B Electrical Engineering' ∧ sex=='SEX_M' ∧ type_vooropleiding=='BUITENL_SL',Model not appropriate,-0.024,0.057,-0.013,4.584,4.222,4.538,0.721820,35,False,False,False
8,croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E' ∧ type_vooropleiding=='BUITENL_SL',Model not appropriate,0.074,-0.119,-0.013,3.979,4.809,4.353,0.120476,19,False,False,False
9,croho=='B Computer Science & Engineering' ∧ sex=='SEX_M' ∧ type_vooropleiding=='BUITENL_SL',Model not appropriate,-0.129,-0.006,-0.003,5.233,4.667,4.649,0.624455,37,False,False,False


In [13]:
# Compare individual subgroup against global model

subgroup_rank = 1

df_individual = df_s[(df_s['model_type'] == 'global') | (df_s['subgroup_rank'] == subgroup_rank)]

cols = ['subgroup_rank', 'model_type', 'description', 'cookD', 'n_train', 'n_test'] + rest_cols

df_individual[cols]

,subgroup_rank,model_type,description,cookD,n_train,n_test,intercept,coef__total_attended_labsessions,pval__total_attended_labsessions,coef__active_minutes,pval__active_minutes,coef__nr_distinct_files_viewed,pval__nr_distinct_files_viewed,coef__total_course_activities,pval__total_course_activities,coef__distinct_days,pval__distinct_days,coef__nr_files_viewed,pval__nr_files_viewed,coef__nr_practice_exams_viewed,pval__nr_practice_exams_viewed,ttest_p,ttest_stat,ttest_p_mean,ttest_stat_mean,ttest_p_mean_global,ttest_stat_mean_global,wilcoxon_p,wilcoxon_stat,subgroup_better_global,subgroup_better_mean,global_better_mean
0,1,subgroup,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=='SEX_M',19.51,161,122,4.779713,0.059493,0.011402,0.000021,0.374080,0.074019,2.725588e-03,0.006153,1.884854e-01,-0.00205,0.900882,-0.036407,6.297257e-02,0.056328,0.008924,0.00069,-3.274801,0.646858,0.377742,0.009122,-2.393091,0.000634,2490.0,True,False,False
15,N/A,global,NaN,NaN,559,374,2.774317,0.064525,0.000076,0.000040,0.040861,0.096937,4.528641e-12,0.016668,6.406497e-10,-0.02897,0.007907,-0.079991,4.967337e-12,0.072741,0.000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
subgroup_linear_models = pd.read_csv("results/subgroup_linear_models1.csv")

In [7]:
subgroup_linear_models.head(20)

,subgroup,n,cookD,term,coef_group,se_group,t_group,p_group,sig_group,coef_global,se_global,t_global,p_global,sig_global
0,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,Intercept,4.779713,0.643772,7.424547,7.365128e-12,***,2.774317,0.350289,7.920070,1.317561e-14,***
1,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,total_attended_labsessions,0.059493,0.023229,2.561096,1.140157e-02,*,0.064525,0.016182,3.987409,7.580384e-05,***
2,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,active_minutes,0.000021,0.000024,0.891465,3.740796e-01,,0.000040,0.000019,2.049756,4.086108e-02,*
3,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,nr_distinct_files_viewed,0.074019,0.024295,3.046732,2.725588e-03,**,0.096937,0.013699,7.076028,4.528641e-12,***
4,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,total_course_activities,0.006153,0.004658,1.320965,1.884854e-01,,0.016668,0.002649,6.291686,6.406497e-10,***
5,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,distinct_days,-0.002050,0.016432,-0.124754,9.008819e-01,,-0.028970,0.010868,-2.665771,7.906636e-03,**
6,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,nr_files_viewed,-0.036407,0.019437,-1.873026,6.297257e-02,.,-0.079991,0.011327,-7.062031,4.967337e-12,***
7,ECTS=='15' ∧ course_repeater==np.False_ ∧ sex=...,161,19.510850,nr_practice_exams_viewed,0.056328,0.021265,2.648857,8.923789e-03,**,0.072741,0.015230,4.776176,2.293433e-06,***
8,ECTS=='15' ∧ course_repeater==np.False_,222,19.391462,Intercept,4.952012,0.566309,8.744366,6.667954e-16,***,2.774317,0.350289,7.920070,1.317561e-14,***
9,ECTS=='15' ∧ course_repeater==np.False_,222,19.391462,total_attended_labsessions,0.055126,0.019189,2.872732,4.479099e-03,**,0.064525,0.016182,3.987409,7.580384e-05,***
